# Task 2 预训练模型

## 前置知识

[GPT3](./Language%20Models%20are%20Few-Shot%20Learners.md)

1. **预训练-微调范式 :**

	- **预训练 (Pre-training):** 理解其本质是在海量无标签文本上进行自监督学习。

	预训练：在进行具体任务前在海量无标签数据上学习通用知识。  
   自监督：无标注，随机遮盖（BERT）或预测下一个词（GPT）。

	- **微调 (Fine-tuning):** 掌握如何在一个预训练好的模型基础上，利用少量有标签的下游任务数据来调整模型参数，使其适配特定任务。

	在预训练模型的基础上，根据少量标注数据训练几轮，小幅度更新（部分）参数。

----

2. **主流预训练模型架构:**
	- 熟悉预训练模型的三种常见架构：Encoder-Only，Decoder-Only 以及 Encoder-decoder。了解他们的特点，常见的应用场景以及经典模型。

- **Encoder-Only（仅编码器）**  
   - 特点：只包含编码器部分，适合处理整段输入，关注上下文理解。  
   - 应用场景：文本分类等。  
   - 经典模型：[BERT](./BERT.md)。

- **Decoder-Only（仅解码器）**  
   - 特点：只包含解码器部分，适合生成式任务，逐步生成输出。  
   - 应用场景：文本生成等。  
   - 经典模型：GPT。

- **Encoder-Decoder（编码器-解码器）**  [Attention](./Attention%20is%20All%20You%20Need.md)
   - 特点：输入由编码器处理，输出由解码器生成，适合输入到输出的映射任务。  
   - 应用场景：机器翻译等。  
   - 经典模型：BART、T5。

----

## 实践任务

1. 基于与**Task1 相同**的数据集，使用 PyTorch 及开源预训练库，使用预训练的**BERT模型**完成文本分类微调，掌握预训练模型的调用与微调核心流程。
2. 比较部分冻结层和全量微调的区别。

### 分词器

In [1]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese') # BERT分词器
# padding=True: pad 到当前 batch 最长
result = tokenizer(["你好，  世界", "我是个比较长的句子。"], padding=True, truncation=True, max_length=10)
print(result)
result = tokenizer("你好。", "你也好。", padding=True, truncation=True, max_length=10)
print(result)

{'input_ids': [[101, 872, 1962, 8024, 686, 4518, 102, 0, 0, 0], [101, 2769, 3221, 702, 3683, 6772, 7270, 4638, 1368, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [101, 872, 1962, 511, 102, 872, 738, 1962, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
{'input_ids': [[101, 872, 1962, 8024, 686, 4518, 102, 0, 0, 0],  # BERT词表ID
               [101, 2769, 3221, 702, 3683, 6772, 7270, 4638, 1368, 102]], 
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],      # 句子类型标记，单句为0，问答为0、1
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0], 
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}      # 注意力掩码，1表示有效token，0表示padding


{'input_ids': [101, 872, 1962, 511, 102, 872, 738, 1962, 511, 102], 
 'token_type_ids': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import pandas as pd
from modelscope.msdatasets import MsDataset


ds = MsDataset.load('chqingMS/game_chat')
train_data = pd.DataFrame([x for x in ds['train']])
val_data = pd.DataFrame([x for x in ds['validation']])
train_labels = train_data['label'].tolist()
val_labels = val_data['label'].tolist()
train_texts = train_data['sentence'].tolist()
val_texts = val_data['sentence'].tolist()
label2id = {l: i for i, l in enumerate(sorted(set(train_labels)))}
train_y = [label2id[l] for l in train_labels]
val_y = [label2id[l] for l in val_labels]

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

class BertDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=30)
        self.labels = torch.tensor(labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = BertDataset(train_texts, train_y)
val_dataset = BertDataset(val_texts, val_y)

model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=len(label2id))
# 冻结 BERT 前六层参数 参数名示例 encoder.layer.3.attention.self.query.weight
for name, param in model.bert.named_parameters():
    if name.startswith("encoder.layer.") and int(name.split(".")[2]) < 6:
        param.requires_grad = False

training_args = TrainingArguments(
    output_dir='./bert_output',
    num_train_epochs=3,
    per_device_train_batch_size=16, # 每个设备上的训练批大小
    per_device_eval_batch_size=16,  # 每个设备上的验证批大小
    eval_strategy='epoch',          # 每个epoch评估一次
    logging_dir='./bert_logs',
    logging_steps=50,               # 每50步记录日志
    save_strategy='epoch',          # 每个epoch保存模型
    load_best_model_at_end=True,    # 训练结束后加载最佳模型
    metric_for_best_model='eval_accuracy',  # 选择最佳模型的评估指标
    report_to=[],  # 关闭wandb等外部日志
)

# 评估函数
def compute_metrics(eval_pred): # 元组 (logits, labels)
    logits, labels = eval_pred  # logits: [batch_size, num_classes]
    preds = logits.argmax(axis=-1)
    from sklearn.metrics import accuracy_score, f1_score
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='macro')
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.508400,0.419306,0.851805,0.507892
2,0.353000,0.431198,0.847187,0.498364
3,0.200700,0.411323,0.878254,0.522433


{'eval_loss': 0.41132330894470215,
 'eval_accuracy': 0.8782535684298909,
 'eval_f1': 0.5224328987926329,
 'eval_runtime': 2.2584,
 'eval_samples_per_second': 1054.745,
 'eval_steps_per_second': 65.977,
 'epoch': 3.0}

| 微调方式         | Epoch | Training Loss | Validation Loss | Accuracy | F1    |
|------------------|-------|---------------|----------------|----------|-------|
| 全量微调         |   1   |    0.481      |    0.390       | 0.868    | 0.517 |
|                  |   2   |    0.266      |    0.419       | 0.879    | 0.520 |
|                  |   3   |    0.153      |    0.428       | 0.891    | 0.530 |
| 冻结前六层编码层   |   1   |   0.508       |   0.419        | 0.852    | 0.508 |
|                  |   2   |   0.353       |   0.431        | 0.847    | 0.498 |
|                  |   3   |   0.201       |   0.411        | 0.878    | 0.522 |